In [ ]:
import gdstk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cytoolz import unique

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.microfluidics_design as microfluidics_design
import paulssonlab.microfluidics_design.fluid as fluid
import paulssonlab.microfluidics_design.fov as fov

In [ ]:
import pint

u = pint._DEFAULT_REGISTRY

mu = 0.7 * u.cP  # dynamic viscosity
nu = mu / (1 * u.g / u.cm**3)  # kinematic viscosity

In [ ]:
camera_dims = {
    "iris15": (4.25, np.array([5056, 2960])),
    "iris9": (4.25, np.array([2960, 2960])),
    "bsi": (6.5, np.array([2048, 2048])),
}
mags = (20, 40)
fov_dims = {}
for camera, (pixel_size, fov_dim) in camera_dims.items():
    for mag in mags:
        fov_dims[f"{camera} {mag}x"] = fov_dim * pixel_size / mag
fov_dims["isim 60x"] = np.array([3200, 3200]) * 56 / 1000

In [ ]:
fov_dims

In [ ]:
def print_flow_info(height, flow_rate, mu, nu, metadata):
    for chip_name, md in metadata.items():
        flow_info = fluid.snake_flow(
            height,
            md,
            length_unit=u.um,
        )
        print(f"{chip_name}:")
        for input_name, input_flow in flow_info.items():
            R = input_flow["R"] * mu
            delta_P = R * flow_rate
            print(f"    {input_name}: {delta_P.to(u.bar):.2f}")
            print(
                f"        flow rate: {flow_rate:.0f~#P} ({flow_rate.to(u.milliliters/u.hour):.1f~#P}, {flow_rate.to(u.milliliters/u.day):.0f~#P})"
            )
            if "flow_nonuniformity" in input_flow:
                print(
                    f"        flow nonuniformity: {input_flow['flow_nonuniformity']:.2f}"
                )
            if "manifold_width" in md:
                L_e = fluid.entrance_length(
                    height,
                    md["manifold_width"] * u.um,
                    flow_rate,
                    nu,
                )
                print(f"        entrance length: {L_e}")

# Designs

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
# chip_dims = np.array([33e3, 18e3])
chip_dims0 = np.array([33e3, 18e3])
# chip_dims0 = np.array([23e3, 13e3])
# chip_dims = np.array([38e3, 19e3])
chip_dims = np.array([40e3, 19e3])
# chip_dims = np.array([33e3, 18e3])

In [ ]:
plt.figure(figsize=(10, 5))
ax = plt.gca()
ax.add_patch(
    plt.Rectangle(-coverslip_dims / 2, *coverslip_dims, fill=False, color="red", lw=2)
)
ax.add_patch(plt.Rectangle(-chip_dims0 / 2, *chip_dims0, fill=False, ls="-."))
# ax.add_patch(plt.Rectangle(-chip_dims1 / 2, *chip_dims1, fill=False, ls="--"))
ax.add_patch(plt.Rectangle(-chip_dims / 2, *chip_dims, fill=False, lw=2))
ax.set_xlim(-1.2 * coverslip_dims[0] / 2, 1.2 * coverslip_dims[0] / 2)
ax.set_ylim(-1.2 * coverslip_dims[1] / 2, 1.2 * coverslip_dims[1] / 2)
ax.set_aspect("equal")

In [ ]:
%%time
metadata = {}
base_params = dict(
    design_func=microfluidics_design.snake,
    dims=chip_dims,
    trench_length=35,
    feeding_channel_width=40,  # 35,
    trench_gap=20,  # 15,
    # trench_width=1.4,
    trench_spacing=2.1,
    trench_margin=0,
    border_margin=0.5e3,
    port_margin=1e3,
    port_radius=750 / 2,
    # port_radius=0,
    port=False,
    registration_marks="qr",
    registration_mark_barcodes=False,
    trenches=True,
    metadata=metadata,
)
manifold_params = {
    **base_params,
    **dict(
        design_func=microfluidics_design.manifold_snake,
        lanes_per_snake=3,
        manifold_input_style="bend-out",
        manifold_split=4,
        manifold_width=50,
        manifold_input_margin=775,  # 2e3,
        manifold_bend_margin=True,  # 0.2e3,
        manifold_bend_radius=0.1e3,
        manifold_margin=100,
    ),
}

wafer_id = 1
split_manifold_chip = dict(trench_width=1.4)
split_poor_manifold_chip = dict(
    manifold_trench_params=[
        dict(trench_width=1.3),
        dict(trench_width=1.3),
        dict(trench_width=1.5),
        dict(trench_width=1.5),
    ]
)

params = [split_manifold_chip, split_manifold_chip, split_poor_manifold_chip]
params = [
    {
        **manifold_params,
        **p,
        "ur_corner_label": f"{wafer_id}.{chip_id}",
    }
    for chip_id, p in enumerate(params, 1)
]
# chip names
chip_names = [
    "Basilisk {ports} FC{feeding_channel_width} L{trench_length} W{trench_width_} TS{trench_spacing}{reg} TG{trench_gap}".format(
        ports=f"S{p['split']}"
        if "split" in p and np.isscalar(p["split"])
        else "S{} LS{}".format(p["manifold_split"], p["lanes_per_snake"]),
        reg=f" M{p.get('mark_size')} ID{p.get('chip_id')}"
        if p.get("registration_mark_barcodes")
        else "",
        trench_width_=p["trench_width"]
        if "trench_width" in p
        else "/".join(
            unique([str(d["trench_width"]) for d in p["manifold_trench_params"]])
        ),
        **p,
    )
    for p in params
]
# make chip cells
chips = [microfluidics_design.chip(name, **p) for p, name in zip(params, chip_names)]
# calculate FOV info and overlay FOVs on chip cells
fov_grids_df = {}
draw_fov_grids = False
center_margins = True
rotate = False
top_shift = -1
for chip_name, chip in zip(chip_names, chips):
    chip_fov_grids_df = fov.get_fov_grids_df(
        fov_dims,
        metadata[chip_name],
        center_margins=center_margins,
        top_shift=top_shift,
    )
    fov_grids_df[chip_name] = chip_fov_grids_df
    if draw_fov_grids:
        # fov_grids_to_draw = chip_fov_grids_df.xs(pd.IndexSlice["iris15 20x",:,:],drop_level=False)
        fov_grids_to_draw = chip_fov_grids_df
        fov.draw_fov_grids(
            chip,
            metadata[chip_name],
            fov_grids_to_draw,
            center_margins=center_margins,
            rotate=rotate,
        )
fov_grids_df = pd.concat(
    fov_grids_df,
    names=["chip"],
)
# lay out wafer, output design
wafer_manifest = "\n".join(
    [f"{wafer_id}.{chip_id}) " + name for chip_id, name in enumerate(chip_names, 1)]
)
wafer_diameter = 3 * u.inch
alignment_mark_position = wafer_diameter / 2 - 0.2 * u.inch
main_cell = microfluidics_design.wafer(
    chips,
    label_right="Basilisk\n JQS/DE/YG 230412",
    label_left=wafer_manifest,
    label=True,
    mask=False,
    chip_dims=chip_dims,
    diameter=wafer_diameter.to(u.um).magnitude,
    alignment_mark_position=alignment_mark_position.to(u.um).magnitude,
)
MAX_POINTS = 2000  # LayoutEditor uses 8191, gdstk default is 199
microfluidics_design.write_gds(
    main_cell, "designs/230412basilisk_s4.gds", max_points=MAX_POINTS
)

In [ ]:
print(wafer_manifest)

In [ ]:
{k: [x["num_trenches"] for x in v["trench_info"].values()] for k, v in metadata.items()}

In [ ]:
{k: v["num_trenches"] for k, v in metadata.items()}

In [ ]:
{k: [x["num_lanes"] for x in v["input_info"].values()] for k, v in metadata.items()}

In [ ]:
{k: v["num_lanes"] for k, v in metadata.items()}

In [ ]:
# Q = 300 * u.uL / u.min
Q = 75 * u.uL / u.min
height = 70 * u.um
print_flow_info(height, Q, mu, nu, metadata)

In [ ]:
fov_grids_df.loc[
    pd.IndexSlice[
        "Basilisk S4 LS3 FC40 L35 W1.4 TS2.1 TG20",
        ["iris15 20x", "iris9 20x", "isim 60x"],
        0,
        :,
    ],
    :,
].round(2)

# Fluids